In [8]:
import numpy as np
import pandas as pd
import os
import regex as re
# !pip install snorkel
# !pip install detoxify
# from transformers import pipeline
# from snorkel.labeling import labeling_function
# from textblob import TextBlob

In [9]:
def split_words_with_colon(data):
    pattern = r'(\w+):(.*)'
    result = []

    pairs = data.split('\t')

    for pair in pairs:
        match = re.match(pattern, pair.strip())

        if match:
            result.append([match.group(1), match.group(2).strip()])

    return result

In [10]:
def make_df(path):
    with open(path) as data:
        sep_data = data.read()
        split_data = split_words_with_colon(sep_data)
        df_sep_data = pd.DataFrame(split_data, columns = ["Author", "Message"])
        df_sep_data['Author_ID'], _ = pd.factorize(df_sep_data['Author'])
        df_sep_data.drop(labels=["Author"],axis=1)
        df_sep_data.drop(labels=["Author"],axis=1)
    return df_sep_data.drop(labels=["Author"],axis=1)

In [22]:
import os


# Specify the folder path
folder_path = '/kaggle/input/discord-data/v3/content/drive/Shareddrives/Datasets/cleaned-v4/discord-v3'

# Create an empty list to store the DataFrames
dfs = []

counter = 0

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a TXT file
    #if filename.endswith('.txt'):
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)
        
        # Read the CSV file into a DataFrame
        df = make_df(file_path)
        
        # Append the DataFrame to the list
        dfs.append(df)
        
        counter +=1
        if counter > 120:
            break

# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Print the combined DataFrame
print(combined_df)

                                                    Message  Author_ID
0                                ral pull the other one too          0
1                                            what other one          1
2         yo\nthe raider\nplease raid @Jinx\nhe said you...          0
3                                                @kemo dad?          2
4                                    how do you use neonite          3
...                                                     ...        ...
16958480                   Leaves time to watch anime after         13
16958481                                               Yeet         13
16958482                                     :neutral_face:         13
16958483  Fuk im tired\n@Lewd Kami Lucifurr lap pillow time         13
16958484          And I'm a beta now\nAnd add my Xbox oppai         13

[16958485 rows x 2 columns]


In [23]:
combined_df.to_csv('combined_df.csv',index=False)

In [ ]:
df_1 = make_df("/kaggle/input/discord-data/v3/content/drive/Shareddrives/Datasets/cleaned-v4/discord-v3/[298954459172700181] [part 10].txt")

In [ ]:
df_1.head()

In [ ]:
df_2 = make_df("/kaggle/input/discord-data/v3/content/drive/Shareddrives/Datasets/cleaned-v4/discord-v3/[510987107087679512] [part 18].txt")
df_3 = make_df("/kaggle/input/discord-data/v3/content/drive/Shareddrives/Datasets/cleaned-v4/discord-v3/[451466119338000405] [part 10].txt")
df_4 = make_df("/kaggle/input/discord-data/v3/content/drive/Shareddrives/Datasets/cleaned-v4/discord-v3/[413225913254412288] [part 2].txt")
df_5 = make_df("/kaggle/input/discord-data/v3/content/drive/Shareddrives/Datasets/cleaned-v4/discord-v3/[360462032811851778] [part 8].txt")

In [ ]:
df_2.head()

In [20]:
df_1.to_csv('disc_1.csv',index=False)
df_2.to_csv('disc_2.csv',index=False)
df_3.to_csv('disc_3.csv',index=False)
df_4.to_csv('disc_4.csv',index=False)
df_5.to_csv('disc_5.csv',index=False)

In [21]:
df_6 = pd.concat([df_1, df_2, df_3, df_4, df_5], ignore_index=True, axis=0)

In [22]:
df_6.head()

,Message,Author_ID
0,FEMBOYS ARE THE GREATEST,0
1,inhales,1
2,OW STOMACH CRAMPS,0
3,guess I need to hide my gender again\nNo\nDun ...,1
4,yo Monx\nhow tf you handle period cramps?,0


In [23]:
df_6.describe()

,Author_ID
count,732863.000000
mean,231.486495
std,473.117436
min,0.000000
25%,6.000000
50%,32.000000
75%,168.000000
max,3032.000000


In [24]:
df_6.to_csv('disc_6.csv',index=False)

In [25]:
df_7 = make_df("/kaggle/input/discord-data/v3/content/drive/Shareddrives/Datasets/cleaned-v4/discord-v3-detox/[360462032811851778] [part 12].txt")

In [26]:
df_7.head()

,Message,Author_ID
0,you didnt wanna play trivia in deletedchannel ...,0
1,C,1
2,Without message,2
3,it was m e,1
4,Scopral is in English one\nwelcome all,0


In [27]:
df_7.to_csv('disc_7.csv',index=False)

In [ ]:
# Define the label mappings for convenience
ABSTAIN = -1
DO_NOT_REMOVE = 0
REMOVE = 1

In [ ]:
def textblob_polarity(x):
    return True if TextBlob(x["Message"]).sentiment.polarity < -0.5 else False

In [ ]:
#df_1_bad=df_1[textblob_polarity(df_1["Message"])==REMOVE]
filtered_df = df_1[df_1.apply(textblob_polarity, axis=1)]

In [ ]:
filtered_df.head()

In [ ]:
df_1.head()

# Labeling functions:
- not used cause we need ground truth!

In [49]:


pos_neg_neu_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")

strongest_emotion_model = pipeline("text-classification", model='bhadresh-savani/distilbert-base-uncased-emotion')
all_emotion_model = pipeline("text-classification", model='bhadresh-savani/distilbert-base-uncased-emotion',
                             return_all_scores=True)
zero_shot_classifier = pipeline("zero-shot-classification",
                                model="facebook/bart-large-mnli")


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [10]:

@labeling_function()
def lf_textblob_polarity(x):
    return DO_NOT_REMOVE if TextBlob(x.text).sentiment.polarity > 0.3 else ABSTAIN


In [ ]:

@labeling_function()
def bad_test(x):
    return DO_NOT_REMOVE if TextBlob(x.text).sentiment.polarity > 0.3

In [68]:
@labeling_function()
def lf_zero_shot(message):
    zero_shot_labels = ["nsfw", "toxic", "racist", "sexist"]
    zero_shot = zero_shot_classifier(message, zero_shot_labels, multi_label=True)
    for label in zero_shot["labels"]:
        if (zero_shot_labels["scores"][zero_shot_labels["labels"].index(label)] > 0.8):
            return REMOVE

    return DO_NOT_REMOVE

In [65]:
@labeling_function()

def lf_emotions(message):
    unwanted_emotions = ["anger"]
    big_emotions = []
    emotions = all_emotion_model(message)[0]
    for emotion in emotions:
            if emotion["score"] > 0.6:
                big_emotions.append(emotion["label"])
            if emotion["score"] == max([a["score"] for a in emotions]):
                biggest_emotion = emotion["label"]
            if emotion["score"] == sorted([a["score"] for a in emotions])[-2]:
                second_biggest_emotion = emotion["label"]
    
    if biggest_emotion in unwanted_emotions:
            return REMOVE
    elif second_biggest_emotion in unwanted_emotions:
            return REMOVE
    else:
        return ABSTAIN

In [11]:
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier

lfs = [lf_textblob_polarity]

applier = PandasLFApplier(lfs)
L_train = applier.apply(df_1)

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=500, log_freq=50, seed=123)
df_train["label"] = label_model.predict(L=L_train, tie_break_policy="abstain")

  0%|          | 1/123199 [00:00<39:31, 51.95it/s]


AttributeError: 'Series' object has no attribute 'text'

In [63]:
df_1.head()

,Message,Author_ID
0,FEMBOYS ARE THE GREATEST,0
1,inhales,1
2,OW STOMACH CRAMPS,0
3,guess I need to hide my gender again\nNo\nDun ...,1
4,yo Monx\nhow tf you handle period cramps?,0
